# Ensemble Learning and Random Forests

*wisdom of the crowd* - an answer to a complex question, aggregated over thousands of random people, in many cases is better than an expert's answer

This is the core idea behind *ensemble learning*. In this method of prediction, we aggregate the predictions of a group of predictors (like classifiers or regressors). The group of predictors is called an *ensemble*.

Consider a single Decision Tree Classifier. If we train a group of singular Decision Trees, each on a random subset of a larger training set, we are performing ensemble learning. To make a prediction then, we would ask each Decision Tree to make a prediction, then return the class that gets the most votes among all the Decision Trees. This ensemble of Decision Trees is called a *Random Forest*.

Popular ensemble methods include: *bagging*, *boosting*, and *stacking*.

## Voting Classifiers

